In [ ]:
'''
Change your code in such a way that all of these are in their respective files:
model
training code
testing code
regularization techniques (dropout, L1, L2, etc)
dataloader/transformations/image-augmentations
misc items like finding misclassified images
'''

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torchsummary import summary

# Let's visualize some of the images
#%matplotlib inline
import matplotlib.pyplot as plt
import argparse

from torch.optim.lr_scheduler import StepLR,OneCycleLR

from train import *
from test import *
from model import *
from plotter import *
from data import *
# from model_group_norm import *
# from model_layer_norm import *

#from parser_args import norm, epochs

In [4]:
norm='bn'
epochs=20

In [5]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)



# train dataloader
train_loader = load_train()

# test dataloader
test_loader = load_test()

CUDA Available? True


In [7]:
# Printing the summary of the model
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Using: ",device)

if norm == 'bn':
	print("Loading Batchnorm Model")
	model = Net().to(device)
elif norm == 'group':
	print("Loading Group Model")
	model = Net_group_norm().to(device)

elif norm == 'layer':
	print("Loading layer Model")
	model = Net_layer_norm().to(device)

#model = Net().to(device)
#model = Net_group_norm().to(device)
#model = Net_layer_norm().to(device)

model.apply(weights_init)
summary(model, input_size=(1, 28, 28))


Using:  cuda
Loading Batchnorm Model
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
           Dropout-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           1,152
              ReLU-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
           Dropout-7           [-1, 16, 24, 24]               0
         MaxPool2d-8           [-1, 16, 12, 12]               0
            Conv2d-9            [-1, 8, 12, 12]             128
           Conv2d-10           [-1, 10, 10, 10]             720
             ReLU-11           [-1, 10, 10, 10]               0
      BatchNorm2d-12           [-1, 10, 10, 10]              20
          Dropout-13           [-1, 10, 10, 10]               0
  

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

scheduler = OneCycleLR(optimizer, max_lr=0.020,epochs=20,steps_per_epoch=len(train_loader))


for epoch in range(epochs):
    print("EPOCH:", epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 1


Loss=0.6363967061042786 Batch_id=468 Accuracy=52.06: 100%|██████████| 469/469 [00:23<00:00, 20.16it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.2832, Accuracy: 9334/10000 (93.34%)

EPOCH: 2


Loss=0.33075740933418274 Batch_id=468 Accuracy=84.72: 100%|██████████| 469/469 [00:21<00:00, 21.73it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0311, Accuracy: 9883/10000 (98.83%)

EPOCH: 19


Loss=0.18580052256584167 Batch_id=468 Accuracy=96.78: 100%|██████████| 469/469 [00:21<00:00, 22.17it/s] 
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0290, Accuracy: 9898/10000 (98.98%)

EPOCH: 20


Loss=0.045082949101924896 Batch_id=468 Accuracy=96.87: 100%|██████████| 469/469 [00:22<00:00, 21.22it/s]



Test set: Average loss: 0.0296, Accuracy: 9893/10000 (98.93%)

